```sql

CREATE TABLE i_v_inventory (

    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(50),
    category VARCHAR(50),
    supplier_name VARCHAR(50),
    unit_price NUMERIC(4,2),
    quantity_in_stock NUMERIC(3),
    units_per_order NUMERIC(3),
    reorder_level SMALLINT,
    expiry_date DATE
);


INSERT INTO i_v_inventory (product_name, category, supplier_name, unit_price, quantity_in_stock, units_per_order, reorder_level, expiry_date)
VALUES 
('Intrafix Safeset', 'infusion set', 'BBraun', 0.90, 100, 100, 10, '2026-11-01'),
('NaCl 0.9% 100ml', 'Fluids', 'BBraun', 1.21,20,20,3,'2027-01-01'),
('NaCl 0.9% 250ml', 'Fluids', 'BBraun', 1.26,10,10,3,'2027-01-01'),
('Vasofix Safety', 'Needle(22G)', 'BBRaun', 0.86,14,50,3,'2027-12-01'),
('Cosmopor I.V.','Dressing for securing IV cannula','Hartmann',0.54,23,50,3,'2028-03-01'),
('Transofix','transferset sterile fluids','BBraun',0.39,30,50,3,'2028-10-01'),
('VitaminC 7.5 gr','ampoule','Pascorbin',1,11.86,10,1,'2025-05-01'),
('ElectrolyteComplete','ampoule','ViktoriaApotheke',17.90,5,1,1,'2025-04-21'),
('Glutathion3000mg','ampoule','ArnikaManufaktur',48.90,6,1,1,'2024-09-21'),
('ATP Konzentrat','ampoule','ArnikaManufaktur',15,14,10,1,'2025-01-10');
```







```sql
-- Create the supplier table
CREATE TABLE supplier_table (
    supplier_id SERIAL PRIMARY KEY,
    supplier_name VARCHAR(50) NOT NULL
);

-- Create the product table
CREATE TABLE product_table (
    product_id SERIAL PRIMARY KEY,
    product_name VARCHAR(50) NOT NULL,
    category VARCHAR(50) NOT NULL,
    supplier_id INT NOT NULL REFERENCES supplier_table(supplier_id),
    unit_price NUMERIC(5,2) NOT NULL CHECK (unit_price >= 0),
    quantity_in_stock NUMERIC(5) NOT NULL CHECK (quantity_in_stock >= 0),
    reorder_level SMALLINT NOT NULL CHECK (reorder_level >= 0),
    expiry_date DATE NOT NULL
);

-- Create the order table
CREATE TABLE order_table (
    order_id SERIAL PRIMARY KEY,
    product_id INT NOT NULL REFERENCES product_table(product_id),
    order_quantity NUMERIC(5) NOT NULL CHECK (order_quantity >= 0),
    order_date DATE NOT NULL
);

-- Insert suppliers into supplier_table
INSERT INTO supplier_table (supplier_name)
VALUES 
('BBraun'),
('Hartmann'),
('Pascorbin'),
('ViktoriaApotheke'),
('ArnikaManufaktur');

-- Insert products into product_table
INSERT INTO product_table (product_name, category, supplier_id, unit_price, quantity_in_stock, reorder_level, expiry_date)
VALUES 
('Intrafix Safeset', 'infusion set', 1, 0.90, 100, 10, '2026-11-01'),
('NaCl 0.9% 100ml', 'Fluids', 1, 1.21, 20, 3, '2027-01-01'),
('NaCl 0.9% 250ml', 'Fluids', 1, 1.26, 10, 3, '2027-01-01'),
('Vasofix Safety', 'Needle(22G)', 1, 0.86, 14, 3, '2027-12-01'),
('Cosmopor I.V.', 'Dressing for securing IV cannula', 2, 0.54, 23, 3, '2028-03-01'),
('Transofix', 'transferset sterile fluids', 1, 0.39, 30, 3, '2028-10-01'),
('VitaminC 7.5 gr', 'ampoule', 3, 11.86, 10, 1, '2025-05-01'),
('ElectrolyteComplete', 'ampoule', 4, 17.90, 5, 1, '2025-04-21'),
('Glutathion3000mg', 'ampoule', 5, 48.90, 6, 1, '2024-09-21'),
('ATP Konzentrat', 'ampoule', 5, 15.00, 14, 1, '2025-01-10');

-- Example inserts into order_table
INSERT INTO order_table (product_id, order_quantity, order_date)
VALUES 
(1, 100, '2024-06-01'),
(2, 20, '2024-06-02');


## example Queries
- list all products with their suppliers
```sql
SELECT 
    p.product_name,
    p.category,
    p.unit_price,
    p.quantity_in_stock,
    s.supplier_name
FROM product_table p
JOIN supplier_table s ON p.supplier_id = s.supplier_id;

-- find products that need to be reordered
SELECT 
    product_name,
    quantity_in_stock,
    reorder_level
FROM 
    product_table
WHERE 
    quantity_in_stock <= reorder_level;

-- total quantity of each product ordered

SELECT p.product_name, SUM(o.order_quantity) AS total_quantity_ordered
FROM order_table o
JOIN product_table p ON o.product_id = p.product_id
GROUP BY p.product_name;

-- List all products that are expiring soon (within the next month)

SELECT 
    product_name,
    category,
    supplier_id,
    unit_price,
    quantity_in_stock,
    reorder_level,
    expiry_date
FROM product_table
WHERE expiry_date <= CURRENT_DATE + INTERVAL '1 month';

-- list all products with the name 'BBraun'
SELECT 
    p.product_name,
    p.category,
    p.unit_price,
    p.quantity_in_stock,
    p.reorder_level,
    p.expiry_date
FROM product_table p
JOIN supplier_table s ON p.supplier_id = s.supplier_id
WHERE s.supplier_name = 'BBraun';


-- Find the most expensive ampoule? 
SELECT 
    product_name,
    unit_price
FROM product_table
WHERE category = 'ampoule'
ORDER BY unit_price DESC
LIMIT 1;

--Find the most expensive product;
SELECT 
    product_name,
    category,
    supplier_id,
    unit_price,
    quantity_in_stock,
    reorder_level,
    expiry_date
FROM product_table
ORDER BY unit_price DESC
LIMIT 1;

--find least expensive product?
SELECT product_name, unit_price FROM product_table ORDER BY unit_price ASC LIMIT 1; 

-- list all products according to price in ascending order
SELECT product_name, unit_price FROM product_table ORDER BY unit_price ASC; 


-- Which ampoules are  expiring within the next 6 months? 
SELECT 
    product_name,
    category,
    supplier_id,
    unit_price,
    quantity_in_stock,
    reorder_level,
    expiry_date
FROM product_table
WHERE category = 'ampoule' AND expiry_date <= CURRENT_DATE + INTERVAL '6 months';





```
